In [1]:
# https://python.langchain.com/docs/use_cases/sql/agents

# Prepare SQL

In [2]:
%%writefile create_db.sql

CREATE TABLE Orders (
    Id INT,
    ProductId INT NOT NULL,
    CustomerId INT NOT NULL
);

CREATE TABLE Products (
    Id INT,
    Name CHAR(255)
);

CREATE TABLE Customers (
    Id INT,
    Name CHAR(255)
);

INSERT INTO Orders(Id, ProductId, CustomerId) VALUES(1, 1, 1);
INSERT INTO Orders(Id, ProductId, CustomerId) VALUES(2, 1, 2);
INSERT INTO Orders(Id, ProductId, CustomerId) VALUES(3, 1, 3);
INSERT INTO Orders(Id, ProductId, CustomerId) VALUES(4, 2, 1);
INSERT INTO Orders(Id, ProductId, CustomerId) VALUES(5, 2, 3);
INSERT INTO Orders(Id, ProductId, CustomerId) VALUES(6, 2, 4);
INSERT INTO Orders(Id, ProductId, CustomerId) VALUES(7, 3, 3);
INSERT INTO Orders(Id, ProductId, CustomerId) VALUES(8, 3, 4);

INSERT INTO Products(Id, Name) VALUES(1, 'pixel6a');
INSERT INTO Products(Id, Name) VALUES(2, 'iphone16');
INSERT INTO Products(Id, Name) VALUES(3, 'xperia1');

INSERT INTO Customers(Id, Name) VALUES(1, 'John');
INSERT INTO Customers(Id, Name) VALUES(2, 'Taro');
INSERT INTO Customers(Id, Name) VALUES(3, 'Ben');
INSERT INTO Customers(Id, Name) VALUES(4, 'Jing');

Overwriting create_db.sql


In [3]:
# https://database.guide/2-sample-databases-sqlite/
!rm orders.sqlite
!sqlite3 orders.sqlite ".read create_db.sql"

In [4]:
import sqlite3

with sqlite3.connect('orders.sqlite') as con:
    response = con.execute("""
    SELECT o.Id, p.Name, c.Name from Orders as o
    INNER JOIN Products as p ON o.ProductId == p.Id
    INNER JOIN Customers as c ON o.CustomerId == c.Id
    """)
    results = response.fetchall()
results

[(1, 'pixel6a', 'John'),
 (2, 'pixel6a', 'Taro'),
 (3, 'pixel6a', 'Ben'),
 (4, 'iphone16', 'John'),
 (5, 'iphone16', 'Ben'),
 (6, 'iphone16', 'Jing'),
 (7, 'xperia1', 'Ben'),
 (8, 'xperia1', 'Jing')]

# Prepare LLM model

In [5]:
import phoenix as px

session = px.launch_app()

from phoenix.trace.langchain import LangChainInstrumentor
LangChainInstrumentor().instrument()

/Users/hide/_project_py312/llm-with-semantic-layer/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📺 To view the Phoenix app in a notebook, run `px.active_session().view()`
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [6]:
from langchain_community.agent_toolkits import create_sql_agent
from langchain_community.llms import Ollama

from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

llm = Ollama(
    # model="llama2:13b-chat",
    model="codellama:7b",
    # callback_manager=callback_manager,
    verbose=True,
)

# Construct agent tool

In [7]:
from langchain.tools import tool
import json

@tool
def get_order_by_product_name(product_name: str) -> str:
    """
    This tool is to fetch the orders with customer name by product name
    """
    with sqlite3.connect('orders.sqlite') as con:
        response = con.execute(f"""
            SELECT o.Id, p.Name, c.Name from Orders as o
            INNER JOIN Products as p ON o.ProductId == p.Id
            INNER JOIN Customers as c ON o.CustomerId == c.Id
            WHERE p.Name == '{product_name}'
        """)
        results = response.fetchall()
        print(f'function called: {results}')
        if results:
            return f"""
{'\n'.join([f'* OrderId: {r[0]}, ProductName: {r[1]}, CustomerName: {r[2]}' for r in results])}
"""
        return "no data"

for attr in ['name', 'description', 'args']:
    print(getattr(get_order_by_product_name, attr))

print(get_order_by_product_name.invoke('pixel6a'))

get_order_by_product_name
get_order_by_product_name(product_name: str) -> str - This tool is to fetch the orders with customer name by product name
{'product_name': {'title': 'Product Name', 'type': 'string'}}
function called: [(1, 'pixel6a', 'John'), (2, 'pixel6a', 'Taro'), (3, 'pixel6a', 'Ben')]

* OrderId: 1, ProductName: pixel6a, CustomerName: John
* OrderId: 2, ProductName: pixel6a, CustomerName: Taro
* OrderId: 3, ProductName: pixel6a, CustomerName: Ben



# Construct agent

In [8]:
from langchain.agents import initialize_agent, AgentType, structured_chat

tools = [get_order_by_product_name]

FORMAT_INSTRUCTIONS = structured_chat.prompt.FORMAT_INSTRUCTIONS + '''
You can refer to the following example of the previous conversation with human.

Example 1:
Human: What is the square root of 4?
Thought: I need to calculate the square root of 4
Action:
```
{{{{
    "action": "calculate_square_root",
    "action_input": {{{{"number": 4}}}}
    
}}}}
```
Observation: {{{{"answer": 2}}}}
Thought: I know what to respond
Action:
```
{{{{
  "action": "Final Answer",
  "action_input": "The square root of 4 is 2"
}}}}
```
<</SYS>>
'''

# from langchain.memory import ConversationBufferWindowMemory
# memory = ConversationBufferWindowMemory(
#     memory_key="chat_history", k=5, return_messages=True, output_key="output"
# )

PREFIX = "[INST]<<SYS>> You are smart that selects a function from list of functions based on user queries.\
Run only one function tool at a time or in one query.<</SYS>>\n"

agent_executor = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    # handle_parsing_errors=True,
    # memory=memory,
    agent_kwargs={
        'prefix': PREFIX, 
    #     'prefix': '[INST]<<SYS>>' + structured_chat.prompt.PREFIX, # + ' DO NOT modify a quoted text in a question.',
        'suffix': structured_chat.prompt.SUFFIX + '\n\n[/INST]',
        'human_message_template': '{input}\n\n{agent_scratchpad}',
        'format_instructions': FORMAT_INSTRUCTIONS
    }
)

/Users/hide/_project_py312/llm-with-semantic-layer/.venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


- https://medium.com/@sandyshah1990/langchain-agents-and-function-calling-using-llama-2-locally-29ce057e4789
- https://github.com/pinecone-io/examples/blob/master/learn/generation/llm-field-guide/llama-2/llama-2-70b-chat-agent.ipynb

In [9]:
agent_response = agent_executor.invoke("Which customers ordered the product 'pixel6a'?")
agent_response



> Entering new AgentExecutor chain...
Action:
```
{
    "action": "get_order_by_product_name",
    "action_input": {"product_name": "pixel6a"}
}
```
function called: [(1, 'pixel6a', 'John'), (2, 'pixel6a', 'Taro'), (3, 'pixel6a', 'Ben')]

Observation: 
* OrderId: 1, ProductName: pixel6a, CustomerName: John
* OrderId: 2, ProductName: pixel6a, CustomerName: Taro
* OrderId: 3, ProductName: pixel6a, CustomerName: Ben

Thought:I know what to respond. Here's my final answer:
```
{
    "action": "Final Answer",
    "action_input": {
        "customer_names": ["John", "Taro", "Ben"]
    }
}
```

> Finished chain.


{'input': "Which customers ordered the product 'pixel6a'?",
 'output': {'customer_names': ['John', 'Taro', 'Ben']}}

# SQL Chain
https://github.com/langchain-ai/langchain/blob/master/cookbook/LLaMA2_sql_chat.ipynb

In [14]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///orders.sqlite")

def get_schema(_):
    return db.get_table_info()


def run_query(query):
    return db.run(query)

print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM Products LIMIT 10;")

db.run("""SELECT c.Name 
FROM Customers AS c 
INNER JOIN Orders AS o ON c.Id = o.CustomerId 
INNER JOIN Products AS p ON p.Id = o.ProductId 
WHERE p.Name = 'pixel6a';
""")

sqlite
['Customers', 'Orders', 'Products']


"[('John',), ('Taro',), ('Ben',)]"

In [32]:
# Prompt
from langchain_core.prompts import ChatPromptTemplate

# Update the template based on the type of SQL Database like MySQL, Microsoft SQL Server and so on
template = """Based on the table schema below, write a SQL query that would answer the user's question:
{schema}

Question: {question}
SQL Query:"""
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Given an input question, convert it to a SQL query without triple backticks. No pre-amble. Do not be ambiguous column name. Preserve table and column names."
        ),
        ("human", template),
    ]
)

# Chain to query
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

sql_response = (
    RunnablePassthrough.assign(schema=get_schema)
    | prompt
    | llm.bind(stop=["\nSQLResult:"])
    | StrOutputParser()
)

sql_response.invoke({"question": "Which customers ordered the product name 'pixel6a'?"})

'```\nSELECT c.Name AS "Customer Name" \nFROM Customers c, Orders o, Products p \nWHERE c.Id = o.CustomerId AND o.ProductId = p.Id AND p.Name LIKE \'%pixel6a%\';\n```'

In [44]:
# Chain to answer
template = """Based on the table schema below, question, sql query, and sql response, write a natural language response:
{schema}

Question: {question}
SQL Query: {query}
SQL Response: {response}"""
prompt_response = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Given an input question, convert it to a SQL query without triple backticks. No pre-amble. Do not be ambiguous column name. Preserve table and column names."
        ),
        ("human", template),
    ]
)

def execute_query(query):
    result = db.run(query["query"].replace("```", ""))
    print(f"""
{query['query']}
{result}
""")
    return result

full_chain = (
    RunnablePassthrough.assign(query=sql_response)
    | RunnablePassthrough.assign(
        schema=get_schema,
        response=execute_query,  #lambda x: db.run(x["query"].replace("```", "")),
    )
    | prompt_response
    | llm
)

# res = full_chain.invoke({"question": "who (customer names) ordered the product 'pixel6a'?"})
res = full_chain.invoke({"question": "what product did John buy?"})
print(res)


SELECT p.Name
FROM Customers c
INNER JOIN Orders o ON c.Id = o.CustomerId
INNER JOIN Products p ON p.Id = o.ProductId
WHERE c.Name = 'John';
[('pixel6a',), ('iphone16',)]


Based on the provided table schema and SQL query, it appears that a customer with the name "John" has purchased two different products from the Orders table. The SQL response is a list of tuples containing the names of the products that John bought, which are 'pixel6a' and 'iphone16'.


Use pre-defined func

In [46]:
from langchain.chains.sql_database.query import create_sql_query_chain

chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "what product did John buy?"})
response

'What product did John buy?\n\nSQLQuery:\nSELECT Products.Name\nFROM Customers, Orders, Products\nWHERE Customers.Id = Orders.CustomerId AND Orders.ProductId = Products.Id AND Customers.Name = "John";'